In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier, StackingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.cluster import KMeans
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow import keras
import pickle
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score, silhouette_score, adjusted_rand_score, normalized_mutual_info_score, rand_score, make_scorer, calinski_harabasz_score
import pickle

In [3]:
data=pd.read_csv("C:\Jupiter\data\smoke (1).csv")

In [4]:
data = data.drop(columns=['UTC'])


In [5]:
y = data["Fire Alarm_Yes"]
X = data.drop(["Fire Alarm_Yes"], axis=1)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=17)

In [7]:
model1 = svm.SVC(kernel='linear')  # Линейное ядро
model1.fit(X_train, y_train)

SVC(kernel='linear')

In [9]:
with open('SVM.pkl', 'wb') as pickle_out:
    pickle.dump(model1, pickle_out)

In [10]:
with open('SVM.pkl', 'rb') as pickle_in: 
    unpickled_list = pickle.load(pickle_in)

In [12]:
model2 = KMeans(n_clusters=3, random_state=42, n_init='auto')
model2.fit(X_train)

KMeans(n_clusters=3, n_init='auto', random_state=42)

In [13]:
with open('kmeans.pkl', 'wb') as pickle_out:
    pickle.dump(model2, pickle_out)

In [14]:
model3 = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)

model3.fit(X_train, y_train)


GradientBoostingClassifier(random_state=42)

In [15]:
with open('GradientBoosting.pkl', 'wb') as pickle_out:
    pickle.dump(model3, pickle_out)

In [16]:
model4 = BaggingClassifier(n_estimators=10, random_state=42)

model4.fit(X_train, y_train)


BaggingClassifier(random_state=42)

In [17]:
with open('Bagging.pkl', 'wb') as pickle_out:
    pickle.dump(model4, pickle_out)

In [18]:
base_classifiers = [
    ('rf', RandomForestClassifier()),
    ('KNN', KNeighborsClassifier()),
    ('SVM',svm.SVC())
]
meta_classifier = LogisticRegression(random_state=42)

model5 = StackingClassifier(estimators=base_classifiers, final_estimator=meta_classifier)
model5.fit(X_train, y_train)


StackingClassifier(estimators=[('rf', RandomForestClassifier()),
                               ('KNN', KNeighborsClassifier()),
                               ('SVM', SVC())],
                   final_estimator=LogisticRegression(random_state=42))

In [19]:
with open('Stacking.pkl', 'wb') as pickle_out:
    pickle.dump(model5, pickle_out)

In [20]:
model6 = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dense(1, activation='sigmoid')

])

model6.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model6.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))




Epoch 1/50


1566/1566 [==============================] - 2s 1ms/step - loss: 7.5213 - accuracy: 0.9555 - val_loss: 8.4059 - val_accuracy: 0.7835
Epoch 2/50
1566/1566 [==============================] - 2s 1ms/step - loss: 2.4871 - accuracy: 0.9643 - val_loss: 3.5050 - val_accuracy: 0.9768
Epoch 3/50
1566/1566 [==============================] - 2s 1ms/step - loss: 2.4440 - accuracy: 0.9657 - val_loss: 2.3429 - val_accuracy: 0.9745
Epoch 4/50
1566/1566 [==============================] - 2s 1ms/step - loss: 1.6953 - accuracy: 0.9746 - val_loss: 2.0297 - val_accuracy: 0.9683
Epoch 5/50
1566/1566 [==============================] - 2s 986us/step - loss: 1.7207 - accuracy: 0.9751 - val_loss: 0.6384 - val_accuracy: 0.9896
Epoch 6/50
1566/1566 [==============================] - 2s 1ms/step - loss: 1.3808 - accuracy: 0.9771 - val_loss: 1.3771 - val_accuracy: 0.9669
Epoch 7/50
1566/1566 [==============================] - 2s 1ms/step - loss: 1.4537 - accuracy: 0.9763 - val_loss: 0.6261 - val_acc

In [21]:
model6.save('Neural.h5')


c:\Python\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
